In [1]:
import numpy as np
from matplotlib import pyplot as plt
import random
import scipy.linalg as spla

def dg(A):
    return A.conj().T

random.seed(1)

In [28]:
# Model parameters.
γ = 1
δ = 0
Ω = 0.5
dΩ = 0.001

# Initial state.
ψ0 = np.array([1, 0], dtype="complex128")

# Simulation parameters.
number_trajectories = 20
t_final = 100
dt = 0.01

In [29]:
# Operators.
σ = np.array([[0,1],[0,0]], dtype="complex128")
c = np.sqrt(γ) * σ
σz = np.array([[1,0], [0,-1]], dtype="complex128")

# Hamiltonian and jump operator.
H = - δ * dg(σ) @ σ + Ω/2. * (dg(σ) + σ)
M1 = np.sqrt(dt) * c

# Displaced operators to compute the derivatives.
Hp = - δ * dg(σ) @ σ + (Ω + dΩ)/2. * (dg(σ) + σ)
Hm = - δ * dg(σ) @ σ + (Ω - dΩ)/2. * (dg(σ) + σ)
M1p = M1
M1m = M1

# The jump operators have to be passed as a list.
M_l = [M1]
Mp_l = [M1p]
Mm_l = [M1m]

In [32]:
def gillespie_fisher(H, Hp, Hm, M_l, Mp_l, Mm_l, dθ, ψ0, t_final, dt, number_trajectories):
    print("Gillespie-Fisher method - Welcome")
    print("=> Starting the pre-computation stage")
    t_range = np.arange(0, t_final, dt)
    
    # Constructs the overall jump operator.
    J = np.zeros_like(M_l[0])
    for M in M_l:
        J += dg(M) @ M
    # Effective (non-Hermitian) Hamiltonian.
    He = H - 1j/2. * J
    
    # Constructs the no-jump evolution operators for all the relevant times.
    V = [] # List of the no-jump evolution operators.
    Qs = [] # List of the non-state-dependent part of the waiting time distribution.
    for t in t_range:
        ev_op = spla.expm(-1j * He * t)
        V.append(ev_op)
        nsd_wtd = dg(ev_op) @ J @ ev_op
        Qs.append(nsd_wtd)
    # Prints the matrix norm of the latest Qs.
    error = spla.norm(Qs[-1])
    print(f"-> Truncation error given by norm of latest Qs matrix -> {error}")
    
    # Displaced versions of the effective Hamiltonian.
    Jp = np.zeros_like(Mp_l[0])
    for Mp in Mp_l:
        Jp += dg(Mp) @ Mp
    Hep = Hp - 1j/2. * Jp
    Jm = np.zeros_like(Mm_l[0])
    for Mm in Mm_l:
        Jm += dg(Mm) @ Mm
    Hem = Hm - 1j/2. * Jm
    
    # Vector of the derivatives of the evolution operator wrt the parameter.
    Vdot = []
    for t in t_range:
        vd = (spla.expm(-1j * Hep * t) - spla.expm(-1j * Hem * t)) / (2. * dθ)
        Vdot.append(vd)
    # Derivatives of all the jump operators.
    Mdot = []
    for n_M in range(len(M_l)):
        Mdot.append((Mp_l[n_M] - Mm_l[n_M]) / (2 * dθ))
    # Precomputation of all the derivatives of MkV(t).
    Δ = []
    for n_M in range(len(M_l)):
        fixed_M = []
        for n_t, t in enumerate(t_range):
            obj = Mdot[n_M] @ V[n_t] + M[n_M] @ Vdot[n_t]
            fixed_M.append(obj)
        Δ.append(fixed_M)
    
    # Evolution cycle.
    print("=> Starting the evolution cycle.")
    
    # List for the results.
    trajectories_results = []
    
    # Cycle over the trajectories.
    for trajectory in range(number_trajectories):
        print(f"-> Starting trajectory {trajectory}")
        
        # Initial state.
        ψ = ψ0
        # Absolute time.
        τ = 0
        # Initial ξ matrix.
        ξ = np.zeros((2,2), dtype="complex128")
        
        results = []
        
        while τ < t_final:
            dict_jump = {}
            
            # Pass to density matrix formalism.
            ρ = np.outer(dg(ψ), ψ)
            
            # Compute the waiting time distribution, exploiting the pre-computed part.
            Ps = []
            for Q in Qs:
                # Even though the number is real, theree might be small imaginary noise.
                wtd = np.real(dg(ψ) @ Q @ ψ)
                Ps.append(wtd)
                
            # Sample from the waiting time distribution.
            n_T = random.choices(range(len(t_range)), weights=Ps)[0]
            # TODO: what happens if T brings us beyond t_final?

            # Increase the absolute time.
            τ += t_range[n_T]
            dict_jump["AbsTime"] = τ
            dict_jump["TimeSinceLast"] = t_range[n_T]

            # Update the state.
            ψ = V[n_T] @ ψ
            # Choose where to jump.
            weights = []
            for M in M_l:
                weight = np.real(dg(ψ) @ dg(M) @ M @ ψ)
                weights.append(weight)
            n_jump = random.choices(range(len(M_l)), weights=weights)[0]
            dict_jump["JumpChannel"] = n_jump
            # Update the state after the jump.
            ψ = M_l[n_jump] @ ψ
            norm = spla.norm(ψ)
            # Renormalize the state.
            ψ = ψ / norm

            print(dict_jump)
                        
            # Compute the ξ matrix.
            ξ = 1./(norm**2) * (M_l[n_jump] @ V[n_T] @ ξ @ dg(V[n_T]) @ dg(M_l[n_jump]))
            ξ += 1./(norm**2) * (Δ[n_jump][n_T] @ ρ @ dg(V[n_T]) @ dg(M_l[n_jump]))
            ξ += 1./(norm**2) * (M_l[n_jump] @ V[n_T] @ ρ @ dg(Δ[n_jump][n_T]))
            dict_jump["ξAfter"] = ξ
            dict_jump["ψAfter"] = ψ
            dict_jump["trξ2"] = (np.real(np.trace(ξ)))**2
        
            results.append(dict_jump)
                
        trajectories_results.append(results)

In [33]:
gillespie_fisher(H, Hp, Hm, M_l, Mp_l, Mm_l, dΩ, ψ0, t_final, dt, number_trajectories)

Gillespie-Fisher method - Welcome
=> Starting the pre-computation stage
-> Truncation error given by norm of latest Qs matrix -> 0.00608198630567942
=> Starting the evolution cycle.
-> Starting trajectory 0
{'AbsTime': 53.07, 'TimeSinceLast': 53.07, 'JumpChannel': 0}
{'AbsTime': 83.9, 'TimeSinceLast': 30.830000000000002, 'JumpChannel': 0}
{'AbsTime': 92.31, 'TimeSinceLast': 8.41, 'JumpChannel': 0}
{'AbsTime': 172.81, 'TimeSinceLast': 80.5, 'JumpChannel': 0}
-> Starting trajectory 1
{'AbsTime': 33.06, 'TimeSinceLast': 33.06, 'JumpChannel': 0}
{'AbsTime': 52.19, 'TimeSinceLast': 19.13, 'JumpChannel': 0}
{'AbsTime': 97.69999999999999, 'TimeSinceLast': 45.51, 'JumpChannel': 0}
{'AbsTime': 155.67, 'TimeSinceLast': 57.97, 'JumpChannel': 0}
-> Starting trajectory 2
{'AbsTime': 65.09, 'TimeSinceLast': 65.09, 'JumpChannel': 0}
{'AbsTime': 84.43, 'TimeSinceLast': 19.34, 'JumpChannel': 0}
{'AbsTime': 127.39000000000001, 'TimeSinceLast': 42.96, 'JumpChannel': 0}
-> Starting trajectory 3
{'AbsTime'